##Setup



In [39]:
!pip install -q pytorch-lightning
!pip install -q transformers

In [40]:
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np

import torch.nn.functional as F
import pytorch_lightning as pl
import torch

from pytorch_lightning.callbacks import ModelCheckpoint

from transformers import AutoModelWithLMHead, AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import math
import random
import re
import argparse
import ast
import io
from tqdm import tqdm, trange

In [41]:
from google.colab import files
uploaded = files.upload()
uploaded1 = files.upload()
uploaded2 = files.upload()

##BART Part

In [42]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'BART/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [43]:
class LitModel(pl.LightningModule):
  # Instantiate the model
  def __init__(self, learning_rate, tokenizer, model, hparams):
    super().__init__()
    self.tokenizer = tokenizer
    self.model = model
    self.learning_rate = learning_rate
    self.hparams = hparams

    if self.hparams.freeze_encoder:
      freeze_params(self.model.get_encoder())

    if self.hparams.freeze_embeds:
      self.freeze_embeds()
  
  def freeze_embeds(self):
    ''' freeze the positional embedding parameters of the model; adapted from finetune.py '''
    freeze_params(self.model.model.shared)
    for d in [self.model.model.encoder, self.model.model.decoder]:
      freeze_params(d.embed_positions)
      freeze_params(d.embed_tokens)

  # Do a forward pass through the model
  def forward(self, input_ids, **kwargs):
    return self.model(input_ids, **kwargs)
  
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr = self.learning_rate)
    return optimizer

  def training_step(self, batch, batch_idx):
    # Load the data into variables
    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]
    # Shift the decoder tokens right (but NOT the tgt_ids)
    decoder_input_ids = shift_tokens_right(tgt_ids, tokenizer.pad_token_id)

    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]
    # Create the loss function
    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    # Calculate the loss on the un-shifted tokens
    loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))
    return {'loss':loss}

  def validation_step(self, batch, batch_idx):

    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]

    decoder_input_ids = shift_tokens_right(tgt_ids, tokenizer.pad_token_id)
    
    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]

    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    val_loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    return {'loss': val_loss}
  
  # Method that generates text using the BartForConditionalGeneration's generate() method
  def generate_text(self, text, eval_beams, early_stopping = True, max_len = 40, startT = None):
    ''' Function to generate text '''
    if startT == None:
      dstartT = self.tokenizer.pad_token_id
    else:
      dstartT = tokenizer.convert_tokens_to_ids(startT)
    generated_ids = self.model.generate(
        text["input_ids"],
        attention_mask=text["attention_mask"],
        use_cache=True,
        decoder_start_token_id = dstartT,
        num_beams= eval_beams,
        max_length = max_len,
        early_stopping = early_stopping
    )
    return [self.tokenizer.decode(w, skip_special_tokens=True, clean_up_tokenization_spaces=True) for w in generated_ids]

def freeze_params(model):
  ''' Function that takes a model as input (or part of a model) and freezes the layers for faster training
      adapted from finetune.py '''
  for layer in model.parameters():
    layer.requires_grade = False


In [44]:
# Create a dataloading module as per the PyTorch Lightning Docs
class SummaryDataModule(pl.LightningDataModule):
  def __init__(self, tokenizer, data_file, batch_size, num_examples = 20000):
    super().__init__()
    self.tokenizer = tokenizer
    self.data_file = data_file
    self.batch_size = batch_size
    self.num_examples = num_examples
  
  # Loads and splits the data into training, validation and test sets with a 60/20/20 split
  def prepare_data(self):
    self.data = pd.read_csv(self.data_file)[:self.num_examples]
    self.train, self.validate, self.test = np.split(self.data.sample(frac=1), [int(.6*len(self.data)), int(.8*len(self.data))])

  # encode the sentences using the tokenizer  
  def setup(self, stage):
    self.train = encode_sentences(self.tokenizer, self.train['source'], self.train['target'])
    self.validate = encode_sentences(self.tokenizer, self.validate['source'], self.validate['target'])
    self.test = encode_sentences(self.tokenizer, self.test['source'], self.test['target'])

  # Load the training, validation and test sets in Pytorch Dataset objects
  def train_dataloader(self):
    dataset = TensorDataset(self.train['input_ids'], self.train['attention_mask'], self.train['labels'])                          
    train_data = DataLoader(dataset, sampler = RandomSampler(dataset), batch_size = self.batch_size)
    return train_data

  def val_dataloader(self):
    dataset = TensorDataset(self.validate['input_ids'], self.validate['attention_mask'], self.validate['labels']) 
    val_data = DataLoader(dataset, batch_size = self.batch_size)                       
    return val_data

  def test_dataloader(self):
    dataset = TensorDataset(self.test['input_ids'], self.test['attention_mask'], self.test['labels']) 
    test_data = DataLoader(dataset, batch_size = self.batch_size)                   
    return test_data



In [45]:
hparams = argparse.Namespace()

hparams.freeze_encoder = True
hparams.freeze_embeds = True
hparams.eval_beams = 4

In [233]:
def shift_tokens_right(input_ids, pad_token_id):
  """ Shift input ids one token to the right, and wrap the last non pad token (usually <eos>).
      This is taken directly from modeling_bart.py
  """
  prev_output_tokens = input_ids.clone()
  index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
  prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
  prev_output_tokens[:, 1:] = input_ids[:, :-1]
  return prev_output_tokens

def encode_sentences(tokenizer, source_sentences, target_sentences, max_length=32, pad_to_max_length=True, return_tensors="pt"):
  ''' Function that tokenizes a sentence 
      Args: tokenizer - the BART tokenizer; source and target sentences are the source and target sentences
      Returns: Dictionary with keys: input_ids, attention_mask, target_ids
  '''

  input_ids = []
  attention_masks = []
  target_ids = []
  tokenized_sentences = {}

  for sentence in source_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim = 0)
  attention_masks = torch.cat(attention_masks, dim = 0)

  for sentence in target_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )
    # Shift the target ids to the right
    # shifted_target_ids = shift_tokens_right(encoded_dict['input_ids'], tokenizer.pad_token_id)
    target_ids.append(encoded_dict['input_ids'])

  target_ids = torch.cat(target_ids, dim = 0)
  

  batch = {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
      "labels": target_ids,
  }

  return batch


def noise_sentence(sentence_, percent_words, replacement_token = "<mask>"):
  '''
  Function that noises a sentence by adding <mask> tokens
  Args: sentence - the sentence to noise
        percent_words - the percent of words to replace with <mask> tokens; the number is rounded up using math.ceil
  Returns a noised sentence
  '''
  # Create a list item and copy
  sentence_ = sentence_.split(' ')
  sentence = sentence_.copy()
  
  num_words = math.ceil(len(sentence) * percent_words)
  
  # Create an array of tokens to sample from; don't include the last word as an option because in the case of lyrics
  # that word is often a rhyming word and plays an important role in song construction
  sample_tokens = set(np.arange(0, np.maximum(1, len(sentence))))
  
  words_to_noise = random.sample(sample_tokens, num_words)
  
  # Swap out words, but not full stops
  for pos in words_to_noise:
      if sentence[pos] != '.':
          sentence[pos] = replacement_token
  
  # Remove redundant spaces
  sentence = re.sub(r' {2,5}', ' ', ' '.join(sentence))
  
  # Combine concurrent <mask> tokens into a single token; this just does two rounds of this; more could be done
  sentence = re.sub(r'<mask> <mask>', "<mask>", sentence)
  sentence = re.sub(r'<mask> <mask>', "<mask>", sentence)
  return sentence
  

In [47]:
# Load the model
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig, BartModel

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base', add_prefix_space=True)

bart_model = BartForConditionalGeneration.from_pretrained(
    "facebook/bart-base")


In [48]:
# Load the data into the model for training
summary_data = SummaryDataModule(tokenizer, '/content/gdrive/My Drive/BART learns to rap/lyrics_simple_noised.csv',
                                 batch_size = 16, num_examples = 140000)

# Load the model from a pre-saved checkpoint; alternatively use the code below to start training from scratch
# model = LitModel.load_from_checkpoint(base_dir + "checkpoint_files_2/8_ep_140k_simple_0210.ckpt",
#                                       learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model, hparams = hparams)

model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model, hparams = hparams)

In [49]:
#checkpoint = ModelCheckpoint(filepath=base_dir + 'checkpoint_files_2/')
trainer = pl.Trainer(gpus = 1,
                     max_epochs = 2,
                     min_epochs = 2,
                     auto_lr_find = False,
                     
                     progress_bar_refresh_rate = 500)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [50]:
# Fit the instantiated model to the data
trainer.fit(model, summary_data)


  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 139 M 
-------------------------------------------------------
139 M     Trainable params
0         Non-trainable params
139 M     Total params
557.682   Total estimated model params size (MB)


1

In [51]:


def generate_lyrics(seed_line, num_lines, model_, startW, noise_percent = 0.25, multiple_lines = False, max_line_history = 3):
  ''' Function that generates lyrics based on previously generated lyrics 
      Args: seed_line - a line to start off the machine
            num_lines - the number of lines to generate
            model_ - the model used to generate the text
            multiple_lines - whether the model generates based on multiple previous lines or just the past line
            max_line_history - the maximum number of previous lines used in the current input
      Returns a list with num_lines of rap lines
  '''
  # Put the model on eval mode
  model_.to(torch.device('cpu'))
  model_.eval()
  lyrics = []
  lyrics.append(seed_line)
  prompt_line_tokens = tokenizer(noise_sentence(seed_line, 0.2), max_length = 32, return_tensors = "pt", truncation = True)
  # Loop through the number of lines generating a new line based on the old

  line = [seed_line]
  for i in range(num_lines):
    # Print out the new line
    print(line[0].strip())
    lyrics.append(line[0])
    line = model.generate_text(prompt_line_tokens, eval_beams = 4, startT = startW)
    # This deals with an artefact in the training data that I had an issue cleaning
    if line[0].find(":") != -1:
      line[0] = re.sub(r'[A-Z]+: ', '', line[0])
    # This allows the model to generate a new line conditioned on more than one line
    if multiple_lines:
      start_line = np.maximum(0, i - max_line_history)
      end_line = i
      prompt_line = ' '.join(lyrics[start_line:end_line]) # Going to end_line is fine because it is non-inclusive
    else:
      prompt_line = lyrics[i]
    prompt_line_tokens = tokenizer(noise_sentence(prompt_line, noise_percent), max_length = 32, return_tensors = "pt", truncation = True)

  return lyrics

# LSTM


In [52]:
BATCH_SIZE = 32
MIN_WORD_FREQUENCY = 10
SEQUENCE_LEN = 5
EPOCH = 6

In [53]:
#open file and save it in to list 
with open('outRlyrics.txt', 'r') as content_file:
    content = content_file.read()
songlyrics = content.split("\"")

#cleaning - removing empty elements from the list
clean_songlyrics = []
for item in songlyrics:
  if len(item)>1:
    clean_songlyrics.append(item)

#make a list(lyrics) of list(sentences)
Llyrics = []
for l in clean_songlyrics:
  sentences = l.split("\n")
  Llyrics.append(sentences)

#cleaning - removing empty elements from sentences of each lyric
Llyrics = [[s for s in l if len(s)>1] for l in Llyrics ]

In [54]:
#cleaning - removing brackets from the main list (Llyrics)
import re
regex = re.compile(".*?\[(.*?)\]")

bracket = []
for i in range(len(Llyrics)):
  for j in range(len(Llyrics[i])):
    result = re.findall(regex, Llyrics[i][j])
    for item in result:
      #keep them in the bracket list
      if item not in bracket:
        bracket.append(item)
      #remove the brackets from the sentences
      Llyrics[i][j] = Llyrics[i][j].replace("["+item+"]", "")
    
#cleaning- remove empty elements (sentences) appeared on each lyric
Llyrics[i] = [s for s in Llyrics[i] if len(s)>1]

for i in range(len(Llyrics)):
  for j in range(len(Llyrics[i])):
    Llyrics[i][j] += "\n"

In [55]:
text_in_words = [w.lower()   for l in Llyrics for s in l for w in s.split(' ') if len(w)>0]

word_freq = {}
for word in text_in_words:
    word_freq[word] = word_freq.get(word, 0) + 1
    
ignored_words = set()
for k, v in word_freq.items():
    if word_freq[k] < MIN_WORD_FREQUENCY:
        ignored_words.add(k)
        
words = set(text_in_words)
words = sorted(set(words) - ignored_words)

word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

In [56]:
STEP = 1

sentences = []
next_words = []
ignored = 0
for i in range(0, len(text_in_words) - SEQUENCE_LEN, STEP):
    # Only add sequences where no word is in ignored_words
    if len(set(text_in_words[i: i+SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
        sentences.append(text_in_words[i: i + SEQUENCE_LEN])
        next_words.append(text_in_words[i + SEQUENCE_LEN])
    else:
        ignored = ignored+1

In [57]:
def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    tmp_sentences = []
    tmp_next_word = []
    for i in np.random.permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))
    return x_train, y_train, x_test, y_test

sentences, next_words, sentences_test, next_words_test = shuffle_and_split_training_set(sentences, next_words)

Size of training set = 792868
Size of test set = 16181


In [58]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding, InputLayer

Lmodel = Sequential()
Lmodel.add(Embedding(input_dim=len(words), output_dim=1024))
Lmodel.add(Bidirectional(LSTM(128)))
Lmodel.add(Dropout(0.2))
Lmodel.add(Dense(len(words)))
Lmodel.add(Activation('softmax'))

In [59]:
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN), dtype=np.int32)
        y = np.zeros((batch_size), dtype=np.int32)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index % len(sentence_list)]):
                x[i, t] = word_indices[w]
            y[i] = word_indices[next_word_list[index % len(sentence_list)]]
            index = index + 1
        yield x, y

In [60]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    examples_file.write('\n----- Generating text after Epoch: %d\n' % epoch)
    # Randomly pick a seed sequence
    seed_index = np.random.randint(len(sentences+sentences_test))
    seed = (sentences+sentences_test)[seed_index]
    
    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed
        examples_file.write('----- Diversity:' + str(diversity) + '\n')
        examples_file.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
        examples_file.write(' '.join(sentence))

        for i in range(50):
            x_pred = np.zeros((1, SEQUENCE_LEN, len(words)))
            for t, word in enumerate(sentence):
                x_pred[0, t, word_indices[word]] = 1.

            preds = Lmodel.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            sentence = sentence[1:]
            sentence.append(next_word)

            examples_file.write(" "+next_word)
        examples_file.write('\n')
    examples_file.write('='*80 + '\n')
    examples_file.flush()

In [34]:
#training model

Lmodel.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
file_path = "./checkpoints/LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-" \
                "loss{loss:.4f}-acc{acc:.4f}-val_loss{val_loss:.4f}-val_acc{val_acc:.4f}" % \
                (len(words), SEQUENCE_LEN, MIN_WORD_FREQUENCY)

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', save_best_only=True)
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
callbacks_list = [checkpoint, print_callback, early_stopping]

examples_file = open("examples.txt", "w")

Lmodel.fit_generator(generator(sentences, next_words, BATCH_SIZE),
                        steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
                        epochs=EPOCH, #100
                        validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),
                        validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/6
24778/24778 [==============================] - 2010s 81ms/step - loss: 6.1611 - accuracy: 0.0930 - val_loss: 5.4885 - val_accuracy: 0.1402
Epoch 2/6
24778/24778 [==============================] - 2379s 96ms/step - loss: 5.4676 - accuracy: 0.1415 - val_loss: 5.3693 - val_accuracy: 0.1496
Epoch 3/6
24778/24778 [==============================] - 2419s 98ms/step - loss: 5.2454 - accuracy: 0.1584 - val_loss: 5.3301 - val_accuracy: 0.1537
Epoch 4/6
24778/24778 [==============================] - 2405s 97ms/step - loss: 5.0887 - accuracy: 0.1704 - val_loss: 5.3068 - val_accuracy: 0.1585
Epoch 5/6
24778/24778 [==============================] - 2405s 97ms/step - loss: 4.9709 - accuracy: 0.1817 - val_loss: 5.2839 - val_accuracy: 0.1608
Epoch 6/6
24778/24778 [==============================] - 2381s 96ms/step - loss: 4.8812 - accuracy: 0.1903 - val_loss: 5.2766 - val_accuracy: 0.1643


In [167]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [255]:
def LSTMgenerate(seeding, prefix):
  seed = seeding
  sentence = prefix
  for i in range(20):
    x_pred = np.zeros((len(seeding), len(words)))
    for t, word in enumerate(seed):
      x_pred[t, word_indices[word]] = 1.
    preds = Lmodel.predict(x_pred, verbose=0)[0]
    next_index = sample(preds,10)
    next_word = indices_word[next_index]
    sentence.append(next_word)
    seed = seed[1:]
    seed.append(next_word)
    if "\n" in next_word:
      break
  return sentence

# Rhyming Part

In [218]:
STRESSES = {'AA1', 'AE1', 'AH1', 'AO1', 'AW1', 'AY1', 'EH1', 'ER1', 'EY1', 'IH1', 'IY1', 'OW1', 'OY1', 'UH1', 'UW1'}


file = open("dict.txt", "r")

contents = file.read()
PHODICT = ast.literal_eval(contents)

file.close()

def isSubList(listA, listB):
    if len(listA) < len(listB):
        return isSubList(listB, listA)
    n = len(listB)
    for start in range(len(listA)-n+1):
        if all(listA[start+i] == listB[i] for i in range(n)):
            return True
    return False

def getRhymes(word):
    sounds = PHODICT[word]
    for index,sound in enumerate(reversed(sounds)):
        if sound in STRESSES:
            ending = sounds[-index-1:]
            break
    yielded = set()
    for wordB, soundsB in PHODICT.items():
        if (ending == soundsB[-index-1:]) and (soundsB not in yielded) and (not isSubList(sounds, soundsB)):
            yielded.add(soundsB)
            yield wordB



#Generation Part

In [212]:
def rhymeFinder(inWord):
  lastWord = inWord
  sameRhyme = []
  for word in getRhymes(lastWord.upper()):
    sameRhyme.append(word.lower())
  outWord = random.choice(sameRhyme)
  return outWord

def reverseSentence(inList):
  words = inList[0].split()
  outStr = ' '.join(reversed(words))
  return outStr

def reverseW(line):
  words = line.split()
  reversed_words = ' '.join(reversed(words))
  return reversed_words

def remove_values_from_list(the_list, val):
   return [value.lower() for value in the_list if value != val]

In [277]:
firstSeed = "You and me forever "
allLyrics = []

BART_sentence = generate_lyrics(seed_line = firstSeed, num_lines = 2, model_ = model, startW = None,
                           noise_percent = 0.5, multiple_lines = False, max_line_history = 1)
allLyrics.append(BART_sentence[-1])

startword = rhymeFinder(BART_sentence[-1].split()[-1])

seed =  BART_sentence[-1].split(" ")
seed = remove_values_from_list(seed, "")
seed.reverse()
prefix =  [startword]

LSoutput = LSTMgenerate(seed, prefix)
LSoutput = ' '.join(LSoutput)
LS_sentence = reverseW(LSoutput)
LS_sentence[0].capitalize()

allLyrics.append(LS_sentence)
print(allLyrics)

You and me forever
You and me forever
[' You and me forever', 'hell whatsoever']


In [270]:
!pip install rake-nltk
from rake_nltk import Rake

r = Rake()
r.extract_keywords_from_sentences(allLyrics)
keywords = r.get_ranked_phrases()

keySentence = ''
for i in keywords:
  keySentence += i
  keySentence += ' '

In [278]:
BART_sentence1 = generate_lyrics(seed_line = keySentence, num_lines = 2, model_ = model, startW = None,
                           noise_percent = 0.25, multiple_lines = False, max_line_history = 1)
allLyrics.append(BART_sentence1[-1])

startword = rhymeFinder(BART_sentence1[-1].split()[-1])

seed =  BART_sentence1[-1].split(" ")
seed = remove_values_from_list(seed, "")
seed.reverse()
prefix =  [startword]

LSoutput = LSTMgenerate(seed, prefix)
LSoutput = ' '.join(LSoutput)
LS_sentence = reverseW(LSoutput)
LS_sentence[0].capitalize()

allLyrics.append(LS_sentence)
print(allLyrics)

slaughterhouse guap lever forever
I'm on a mission to get the fuck out of here
[' You and me forever', 'hell whatsoever', " I'm on a mission to get the fuck out of here", "ass guessed western rip sour helicopter feeling again? ends loop badge riches spittin' ourselves do? thirty fall pajamas dancer swimming tier"]
